In [1]:
from transformers import BioGptTokenizer
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")


In [6]:
special_tokens_dict = {"clinical_s_token": "<CLINICAL>"}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

AssertionError: Key clinical_s_token is not a special token

In [ ]:
sep_tokens = [
    "<IMG>",
    "</IMG>",
    "<REPORT>",
    "</REPORT>",
    "<CLINICAL>",
    "</CLINICAL>",
    "<CHEXPERT>",
    "</CHEXPERT>"
    "<BB>"
    "</BB>"
]

tokenizer.add_tokens()

In [10]:
tokenizer("that's a good day")

{'input_ids': [2, 22, 121, 14, 746, 262], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [7]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [1]:
from dataset.reflacx import REFLACXDataset
from torch.utils.data import DataLoader
import torch
from model.encoders import *

# import warnings
# import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.filterwarnings("ignore")

In [2]:
d = REFLACXDataset(
    df_path="./spreadsheets/reflacx_clinical.csv",
    mimic_eye_path="F:\\mimic-eye",
    image_size=128,
    split_str="train",
)

In [3]:
loader = DataLoader(
    d,
    batch_size=4,
    shuffle=True,
    collate_fn= lambda x: tuple(zip(*x)),
    drop_last=False,
)

In [4]:
data = next(iter(loader))

c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\dataset\reflacx.py:213: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(data[self.chexpert_label_cols]) == 1
c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\dataset\reflacx.py:231: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(data[self.clinical_labels], dtype=torch.float32)
c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\dataset\reflacx.py:213: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

In [5]:
model = OneDCoreModel(
    hidden_dim=128,
    out_dim=len(loader.dataset.clinical_labels),
)

In [7]:
out = model(data[0])

c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\model\encoders.py:238: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return out[torch.arange(n), torch.tensor(lengths).long() - 1, :]


In [8]:
out.shape

torch.Size([4, 6])

In [9]:
data[0]

({'xray': tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0510, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0431, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0627, 0.0549,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0627, 0.0588,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0706, 0.0627,  ..., 0.0000, 0.0000, 0.0000]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0510, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0431, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0627, 0.0549,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0627, 0.0588,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0706, 0.0627,  ..., 0.0000, 0.0000, 0.0000]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0510, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
           [0.

In [5]:
modalities = ['xray', 'bb_label', 'clinical_data', 'chexpert_label', 'report']

batch_map = {
    m: [ d[m] for d in data[0]]
    for m in modalities
}

In [7]:
imgs = torch.stack(batch_map['xray'])

In [8]:
img_enc = ImageEncoder(hidden_dim=128)

In [9]:
img_out = img_enc(imgs)

In [10]:
img_out.shape

torch.Size([4, 128])

In [11]:
import dataset.constants as r_constants

In [12]:
bb_enc = BBEncoder(
    hidden_dim=128,
    nhead=4,
    nlayer=4,
    emb_dim=32,
    label_cols=r_constants.TOP5_LABEL_COLS,
)

In [13]:
bb_out = bb_enc(batch_map['bb_label'])

In [14]:
bb_out.shape

torch.Size([4, 128])

In [15]:
batch_map['clinical_data']

[{'cat': {'gender': tensor(0, dtype=torch.int32)}, 'num': tensor([-1.7269])},
 {'cat': {'gender': tensor(0, dtype=torch.int32)}, 'num': tensor([-0.5909])},
 {'cat': {'gender': tensor(0, dtype=torch.int32)}, 'num': tensor([1.1401])},
 {'cat': {'gender': tensor(0, dtype=torch.int32)}, 'num': tensor([-0.8614])}]

In [16]:
clinical_enc = FCCatEncoder(
    num_dim=1,
    hidden_dim=128,
    n_layers=5,
)

In [17]:
clinical_out = clinical_enc(batch_map['clinical_data'])

In [18]:
clinical_out.shape

torch.Size([4, 128])

In [19]:
batch_map['chexpert_label']

[tensor([False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False]),
 tensor([False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False]),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False]),
 tensor([False, False, False, False, False, False, False, False,  True, False,
         False, False, False,  True])]

In [20]:
chexpert_enc = FCEncoder(
    hidden_dim=128,
    input_dim=14,
    n_layers=5,
)

In [21]:
chexpert_out = chexpert_enc(torch.stack(batch_map['chexpert_label']).float())

In [22]:
chexpert_out.shape

torch.Size([4, 128])

In [23]:
report_enc = ReportEncoder(hidden_dim=128, n_heads=4, n_layers=4)

In [24]:
from transformers import BioGptTokenizer


In [29]:
report_out = report_enc(
    batch_map['report'],
)

c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\model\encoders.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return out[torch.arange(n), torch.tensor(lengths).long() - 1, :]


In [30]:
report_out.shape

torch.Size([4, 128])